Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

# Make a node

In [2]:
μ = rand(5, 5)
σ = rand(5, 5)
aa = RandomNode(:a, [:i, :j], Normal, μ, σ)
bb = RandomNode(:b, [:j, :k], Gamma, rand(5, 3), rand(5, 3))

VB.RandomNode{Distributions.Gamma}(:b,Symbol[],[:j,:k],5x3 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=0.9434406323473687, θ=0.14756700584199822)   …  Distributions.Gamma(α=0.9056199266348519, θ=0.574828935152828)   
 Distributions.Gamma(α=0.9689460438400113, θ=0.3190963679861831)       Distributions.Gamma(α=0.1817685209930895, θ=0.9808739049867006)  
 Distributions.Gamma(α=0.9392798645505624, θ=0.21077339116585891)      Distributions.Gamma(α=0.20278919214839775, θ=0.47906766924878275)
 Distributions.Gamma(α=0.6430772007885319, θ=0.21555695269065422)      Distributions.Gamma(α=0.2085796033274807, θ=0.9943555987905646)  
 Distributions.Gamma(α=0.29067948024865964, θ=0.12893931087508648)     Distributions.Gamma(α=0.30721602758768296, θ=0.3230012748848814) )

In [3]:
cc = ConstantNode(:c, [:i, :j], rand(3, 4))

VB.ConstantNode{Float64}(:c,Symbol[],[:i,:j],3x4 Array{Float64,2}:
 0.914171  0.0216755  0.107728  0.67686 
 0.427475  0.582149   0.984993  0.7092  
 0.485808  0.732233   0.621352  0.111843)

In [4]:
cc = ConstantNode(rand(3, 4), [:i, :j])

VB.ConstantNode{Float64}(symbol("##const#7735"),Symbol[],[:i,:j],3x4 Array{Float64,2}:
 0.0299368  0.53131   0.55386   0.0350722
 0.498049   0.293163  0.792412  0.828048 
 0.830429   0.313445  0.841221  0.863572 )

In [5]:
nodes = Node[aa, bb]
fi = get_structure(nodes...)
fi

VB.FactorInds([:i,:j,:k],[5,3,5],Dict(:a=>[1,2],:b=>[2,3]),Dict(:a=>[1,2],:b=>[1,2]))

In [6]:
a[i, j] ~ Normal(μ, σ)

VB.RandomNode{Distributions.Normal}(:a,Symbol[],[:i,:j],5x5 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=0.8442194039363558, σ=0.9323780095864198)   …  Distributions.Normal(μ=0.8680633789889216, σ=0.17081250549221072)
 Distributions.Normal(μ=0.7874535451748825, σ=0.20018721613127166)     Distributions.Normal(μ=0.15064338651266174, σ=0.5859067337693438)
 Distributions.Normal(μ=0.7437323836260663, σ=0.6457670718819153)      Distributions.Normal(μ=0.4089503338656877, σ=0.597057067671674)  
 Distributions.Normal(μ=0.5859191580536203, σ=0.27512951776105155)     Distributions.Normal(μ=0.6699764950916043, σ=0.5234763123293376) 
 Distributions.Normal(μ=0.5583784063702517, σ=0.5695621135439943)      Distributions.Normal(μ=0.6809501622282659, σ=0.4422602772248374) )

In [7]:
c[i, j] ~ Const(rand(3, 4))

VB.ConstantNode{Float64}(:c,Symbol[],[:i,:j],3x4 Array{Float64,2}:
 0.717459  0.386429  0.959291   0.945281
 0.10788   0.608984  0.3555     0.701513
 0.270013  0.35235   0.0308718  0.759846)

Make a multivariate node

In [8]:
dims = (5, 3)
m = [rand(dims[1]) for x in 1:dims[2]]
VV = [diagm(rand(dims[1])) for x in 1:dims[2]]
d[i, j] ~ MvNormal(m, VV)
d.innerinds, d.outerinds

([:i],[:j])

# Make a factor

In [9]:
dims = (10, 2)

μ[j] ~ Normal(rand(dims[2]), ones(dims[2]))
τ ~ Gamma(1, 1)
x[i, j] ~ Normal(rand(dims), ones(dims))

f = @factor LogNormalFactor x μ τ;
value(f)

-35.199017273048135

and a multivariate factor...

In [10]:
dims = (5, 3)
m = [rand(dims[1]) for x in 1:dims[2]]
VV = [diagm(rand(dims[1])) for x in 1:dims[2]]

y[i, j] ~ MvNormal(m, VV)
μ[i] ~ Normal(zeros(dims[1]), ones(dims[1]))
Σ[j] ~ Const([diagm(rand(dims[1])) for x in 1:dims[2]])

f = @factor LogMvNormalCanonFactor y μ Σ
value(f)

-28.915894683085995

In [11]:
vars = fieldnames(f)
val_expr = value(typeof(f))
using Base.Cartesian
macroexpand(:(@wrapvars $vars $val_expr (@ntuple 1 i)))

quote  # /Users/jmxp/code/vbgraph/types.jl, line 353:
    δ = E(project(f,:x,(i_1,))) - E(project(f,:μ,(i_1,))) # /Users/jmxp/code/vbgraph/types.jl, line 354:
    -(1 / 2) * ((trace(E(project(f,:Λ,(i_1,))) * (V(project(f,:x,(i_1,))) + V(project(f,:μ,(i_1,))) + δ * δ')) + length(project(f,:x,(i_1,))) * log(2π)) - Elogdet(project(f,:Λ,(i_1,))))
end

In [12]:
f.inds.indices, f.inds.inds_in_factor, f.inds.inds_in_node

([:j],Dict(:μ=>Int64[],:y=>[1],:Σ=>[1]),Dict(:μ=>Int64[],:y=>[1],:Σ=>[1]))

In [13]:
project_inds(f, :x, (5,)), project_inds(f, :μ, (5,))

(Any[5],Any[Colon()])

In [14]:
μ

VB.RandomNode{Distributions.Normal}(:μ,Symbol[],[:i],[Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0)])

In [15]:
g = @factor EntropyFactor x
value(g)

28.378770664093462

# Let's make a simple model

We want a simple model of inference for a normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\mu, \tau) \\
    \mu \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

## Make the nodes

We need to make nodes for all the random variables that will need to be updated (we can also create nodes for parameter arrays, but these will be converted automatically if we don't).

In [16]:
dims = (20, 6)

# note: it won't matter much how we initialize here
μ[j] ~ Normal(zeros(dims[2]), ones(dims[2]))
τ[j] ~ Gamma(1.1 * ones(dims[2]), ones(dims[2]))
μ0[j] ~ Const(zeros(dims[2]))
τ0[j] ~ Const(2 * ones(dims[2]))
a0[j] ~ Const(1.1 * ones(dims[2]))
b0[j] ~ Const(ones(dims[2]))

y[i, j] ~ Const(rand(dims));

Now make factors: We need a Normal factor for the observation model plus a prior and an entropy for each node.

In [17]:
obs = @factor LogNormalFactor y μ τ
μ_prior = @factor LogNormalFactor μ μ0 τ0
τ_prior = @factor LogGammaFactor τ a0 b0

VB.LogGammaFactor{1}(VB.RandomNode{Distributions.Gamma}(:τ,Symbol[],[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:a0,Symbol[],[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,Symbol[],[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.FactorInds([:j],[6],Dict(:τ=>[1],:b0=>[1],:a0=>[1]),Dict(:τ=>[1],:b0=>[1],:a0=>[1])),Dict(:τ=>:x,:b0=>:β,:a0=>:α))

In [18]:
value(obs), value(μ_prior), value(τ_prior)

(-169.83348910132688,-13.593072740907873,0.0449816833123928)

In [19]:
m = VBModel([μ, τ, μ0, τ0, a0, b0, y], [obs, μ_prior, τ_prior])

VB.VBModel(VB.Node[VB.RandomNode{Distributions.Normal}(:μ,Symbol[],[:j],[Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0)]),VB.RandomNode{Distributions.Gamma}(:τ,Symbol[],[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:μ0,Symbol[],[:j],[0.0,0.0,0.0,0.0,0.0,0.0]),VB.ConstantNode{Float64}(:τ0,Symbol[],[:j],[2.0,2.0,2.0,2.0,2.0,2.0]),VB.ConstantNode{Float64}(:a0,Symbol[],[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,Symbol[],[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.ConstantNode{Float64}(:y,Symbol[],[:i,:j],20x6 Array{Float64,2}:
 0.294129   0.935407   0.794249    0.282745   0.713034    0.904565 
 0.413497   0.479329   0.891217    0.519

In [20]:
[n.name for n in m.nodes]

7-element Array{Any,1}:
 :μ 
 :τ 
 :μ0
 :τ0
 :a0
 :b0
 :y 

In [21]:
[typeof(f) for f in m.factors]

3-element Array{DataType,1}:
 VB.LogNormalFactor{2}
 VB.LogNormalFactor{1}
 VB.LogGammaFactor{1} 

In [22]:
m.graph

Dict{VB.Node,Array{Tuple{VB.Factor{N},Symbol},1}} with 7 entries:
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…

In [23]:
naturals(obs, μ)

6-element Array{Tuple{Float64,Float64},1}:
 (11.526439438927746,-11.000000000000002)
 (10.130185538555713,-11.000000000000002)
 (9.797481715522144,-11.000000000000002) 
 (11.839107934768258,-11.000000000000002)
 (8.669035316940196,-11.000000000000002) 
 (9.35807468380877,-11.000000000000002)  

In [24]:
naturals(obs, τ)

6-element Array{Tuple{Float64,Float64},1}:
 (10.0,13.210305374125424)
 (10.0,12.96247456893213) 
 (10.0,13.024886447820055)
 (10.0,13.531093020809829)
 (10.0,11.988654551161423)
 (10.0,12.542732892997265)

In [25]:
check_conjugate(τ, m)

true

In [26]:
check_conjugate(μ, m)

true

In [27]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)

In [28]:
τ.data

6-element Array{Distributions.Gamma,1}:
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)

In [29]:
update!(m)

In [30]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=0.372248897149849, σ=0.7071067811865476)  
 Distributions.Normal(μ=-0.6150517040613815, σ=0.7071067811865476)
 Distributions.Normal(μ=-0.8503088334551071, σ=0.7071067811865476)
 Distributions.Normal(μ=0.593338910822073, σ=0.7071067811865476)  
 Distributions.Normal(μ=-1.6482409340979403, σ=0.7071067811865476)
 Distributions.Normal(μ=-1.1610165252806863, σ=0.7071067811865476)

In [31]:
τ.data

6-element Array{Distributions.Gamma,1}:
 Distributions.Gamma(α=16.69535745277696, θ=0.8999999999999999) 
 Distributions.Gamma(α=28.40953135375355, θ=0.8999999999999999) 
 Distributions.Gamma(α=33.82866961426824, θ=0.8999999999999999) 
 Distributions.Gamma(α=16.66560055438292, θ=0.8999999999999999) 
 Distributions.Gamma(α=58.145326199996056, θ=0.8999999999999999)
 Distributions.Gamma(α=41.89948966066639, θ=0.8999999999999999) 

# A Multivariate Model

We want a simple model of inference for a multivariate normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\pmb{\mu}, \Sigma) \\
    \mu_i \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.